In [1]:
try:
  import graph_tool
except:
  print('Install graph-tool')
  !echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
  !apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
  !apt-get update
  !apt-get install python3-graph-tool python3-cairo python3-matplotlib

In [2]:
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np

np.set_printoptions(suppress=True)


import os
root = '/content/drive/MyDrive/TransportNet-vladi/Stable Dynamic & Beckman'
import sys 
if root not in sys.path:
  sys.path.append(root)
try:
  import scanf
except:
  !pip install scanf
import os
import data_handler as dh
import sinkhorn as skh
from sinkhorn import Sinkhorn
import model as md
import csv

from conf import parsers

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)


In [3]:
nodes_name = None

best_sink_beta = 0.001
sink_num_iter, sink_eps = 25000, 10 ** (-8)
INF_COST = 100
INF_TIME = 1e10
num_of_iter = 15


def get_times_inverse_func(capacity, times, rho=0.15, mu=0.25):
    capacities = capacity.to_numpy()
    freeflowtimes = times
    return np.transpose((capacities / rho) * (np.power(times / freeflowtimes, mu) - 1.0))


def get_LW(L_dict, W_dict, new_to_old):
    L = np.array([L_dict[new_to_old[i]] for i in range(len(L_dict))], dtype=np.double)
    W = np.array([W_dict[new_to_old[i]] for i in range(len(W_dict))], dtype=np.double)
    people_num = L.sum()
    L /= np.nansum(L)
    W /= np.nansum(W)
    return L, W, people_num

import shutil

def main(default_skh, reduce_L, lr, sfx, rewrite = False):
    # rm_path = os.path.join(root, 'KEV_res')
    # if os.path.exists(rm_path):
    #   shutil.rmtree(rm_path)
    handler = dh.DataHandler()
    graph_data = handler.GetGraphData(eval(f'handler.{parsers}_net_parser'),
                                      columns=['init_node', 'term_node', 'capacity', 'free_flow_time'])

    L_dict, W_dict = handler.GetLW_dicts(eval(f'handler.{parsers}_corr_parser'))
    # handler.save_input_data_to_res(graph_data, L_dict, W_dict, root = os.path.join(root, 'KEV_res/input_data/'))

    handler = dh.DataHandler()

    max_iter = 5
    alpha = 0.9

    empty_corr_dict = {source: {'targets': list(W_dict.keys())} for source in L_dict.keys()}
    empty_corr_matrix, old_to_new, new_to_old = handler.reindexed_empty_corr_matrix(empty_corr_dict)
    print('fill correspondence_matrix')

    L, W, people_num = get_LW(L_dict, W_dict, new_to_old)
    total_od_flow = people_num

    print(f'L, W, people_num {L, W, people_num}, total_od_flow: {total_od_flow}')
    model = md.Model(graph_data, empty_corr_dict, total_od_flow, mu=0.25)
    T_dict = handler.get_T_from_t(graph_data['graph_table']['free_flow_time'],
                                  graph_data, model)
    T = handler.T_matrix_from_dict(T_dict, empty_corr_matrix.shape, old_to_new)

    if default_skh:
      mode = 'default'
      suffix = 'multi'
      cut_L_f = lambda x: x/2
    else:
      mode = 'fast'
      suffix = f'fast_multi_{lr}_'
      suffix = suffix + 'T' if reduce_L else suffix + 'F'
      if reduce_L:
        cut_L_f = lambda x: x/2
      else:
        cut_L_f = lambda x: x
    root_save = os.path.join(root, f'KEV_res/{suffix}' + sfx)
    root_save2 = os.path.join(root, 'res', suffix + sfx)
    log_file = os.path.join(root_save2, 'log.txt')
    log_file2 = os.path.join(root_save2, 'log2.txt')
    if os.path.exists(log_file) and not rewrite:
        return None
    os.makedirs(os.path.dirname(log_file), exist_ok = True)
    with open(log_file, 'w') as f:
        print('Start', file = f)
    os.makedirs(root_save, exist_ok = True)
    for ms_i in range(num_of_iter):
        print('iteration: ', ms_i)
        with open(log_file, 'a') as f:
            print('iteration: ', ms_i, file = f)
        s = None
        s = Sinkhorn(L, W, people_num, sink_num_iter, sink_eps, cut_L_f = cut_L_f, L0 = lr, log_file = log_file)

        T = np.nan_to_num(T, nan=0, posinf=0, neginf=0)

        cost_matrix = np.nan_to_num(T * best_sink_beta, nan=INF_COST)

        with open(log_file2, 'a') as f:
          print('cost matrix', cost_matrix, file = f)
        rec, _, _ = s.iterate(cost_matrix, mode = mode)
        
        with open(log_file2, 'a') as f:
          print('rec', rec, np.sum(rec), file = f)
        sink_correspondences_dict = handler.corr_matrix_to_dict(rec, new_to_old)

        L_new = np.nansum(rec, axis=1)
        L_new /= np.nansum(L_new)
        W_new = np.nansum(rec, axis=0)
        W_new /= np.nansum(W_new)

        model.refresh_correspondences(graph_data, sink_correspondences_dict)
        for i, eps_abs in enumerate(np.logspace(1, 3, 1)):
            solver_kwargs = {'eps_abs': eps_abs,
                            'max_iter': max_iter}

            result = model.find_equilibrium(solver_name='ustm', composite=True,
                                            solver_kwargs=solver_kwargs,
                                            base_flows=alpha * graph_data['graph_table']['capacity'])
              # print(results)

        model.graph.update_flow_times(result['times'])

        # print(result.keys(), np.shape(result['flows']))
        # for flow, time in zip(result['flows'], result['times']):
        #     print('flow, time: ', flow, time)

        T_dict = result['zone travel times']
        T = handler.T_matrix_from_dict(T_dict, rec.shape, old_to_new)
        flows_inverse_func = get_times_inverse_func(graph_data['graph_table']['capacity'], result['times'], rho=0.15,
                                                    mu=0.25)

        subg = result['subg']

        def check_save(path, *args, **kwargs):
          os.makedirs(os.path.dirname(path), exist_ok = True)
          np.savetxt(path, *args, **kwargs)
        check_save(os.path.join(root_save, 'flows/') + str(ms_i) + '_flows.txt', result['flows'], delimiter=' ')
        check_save(os.path.join(root_save, 'times/') + str(ms_i) + '_time.txt', result['times'], delimiter=' ')
        check_save(os.path.join(root_save, 'corr_matrix/') + str(ms_i) + '_corr_matrix.txt', rec, delimiter=' ')
        check_save(os.path.join(root_save, 'inverse_func/') + str(ms_i) + '_inverse_func.txt', flows_inverse_func,
                    delimiter=' ')
        check_save(os.path.join(root_save, 'subg/') + str(ms_i) + '_nabla_func.txt', subg, delimiter=' ')

if __name__ == '__main__':
  # default_skh, reduce_L, lr, sfx
    sfxs = ['']
    # exps = [[False, reduce_L, lr, sfx] for reduce_L in [True, False] for lr in [1, 10, 16, 32] for sfx in sfxs] + [[True, None, 1, sfx] for sfx in sfxs]
    # exps = [[False, True, None, '']]
    exps = [[True, True, 1, '']] + [[False, False, lr, ''] for lr in [8, 2]]
    # exps = 
    for exp in exps:
        main(*exp, rewrite = True)

fill correspondence_matrix
L, W, people_num (array([0.33333333, 0.33333333, 0.33333333]), array([0.33333333, 0.33333333, 0.33333333]), 12000.0), total_od_flow: 12000.0
iteration:  0
number of iterations in Sinkhorn:18
L evolution: []
Phi: 1.7900932471020061
calculating pred to edges
Composite optimization...
Oracles created...
Universal similar triangles method...
L_value, A, alpha 29393.876913398133 None 3.402069087198859e-05
A, alpha 3.402069087198859e-05 3.402069087198859e-05
L_value, A, alpha 14696.938456699067 3.402069087198859e-05 9.294625597086755e-05
A, alpha 0.00012696694684285613 9.294625597086755e-05
L_value, A, alpha 7348.469228349533 0.00012696694684285613 0.00021605368740682977
A, alpha 0.0003430206342496859 0.00021605368740682977
L_value, A, alpha 3674.2346141747666 0.0003430206342496859 0.0004705628229873134
A, alpha 0.0008135834572369992 0.0004705628229873134
iteration:  1
number of iterations in Sinkhorn:18
L evolution: []
Phi: 1.7900499277404633
Composite optimizatio